# Classification model using Multilayer Perceptron

### 1. Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import os

### 2. Upload RNA data

In [4]:
os.chdir("../../Data")

In [7]:
data = pd.read_csv("clinic_and_RNA_data_raw_NIVOLUMAB.csv")
data.head()

,Unnamed: 0,SUBJID,Cohort,Arm,MAF_Tumor_ID,MAF_Normal_ID,CNV_ID,RNA_ID,CD8_IF_ID,Sex,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0,RCC10-20,CM-010,NIVOLUMAB,NaN,NaN,NaN,G138701_RCCBMS-00020-T_v1_RNA_OnPrem,0001 00020,Male,...,33.26816,30.04056,30.36292,30.98129,32.38442,28.01561,31.24379,33.81903,33.22469,31.23383
1,1,RCC10-97,CM-010,NIVOLUMAB,RP-1458_RCCBMS-00097-T_v4_Exome_OnPrem,RP-1458_RCCBMS-00097-N_v1_Exome_OnPrem,RCCBMS-00097-T.called.seg,G138701_RCCBMS-00097-T_v1_RNA_OnPrem,0001 00097,Female,...,31.53633,29.94062,30.80829,30.74475,32.09470,26.32526,32.70057,33.67262,33.10198,32.44643
2,2,RCC10-141,CM-010,NIVOLUMAB,RP-1458_RCCBMS-00141-T_v1_Exome_OnPrem,RP-1458_RCCBMS-00141-N_v1_Exome_OnPrem,RCCBMS-00141-T.called.seg,G138701_RCCBMS-00141-T_v1_RNA_OnPrem,0014 00141,Female,...,30.74474,31.95812,30.73898,30.15898,31.69473,26.33098,32.68330,33.96047,32.97135,32.20607
3,3,RCC10-99,CM-010,NIVOLUMAB,NaN,NaN,NaN,G138701_RCCBMS-00099-T_v1_RNA_OnPrem,0015 00099,Male,...,31.63196,30.71732,31.09654,30.25757,31.90774,27.53819,32.64194,34.54254,33.24885,32.78278
4,4,RCC10-163,CM-010,NIVOLUMAB,RP-1458_RCCBMS-00163-T_v1_Exome_OnPrem,RP-1458_RCCBMS-00163-N_v1_Exome_OnPrem,RCCBMS-00163-T.called.seg,G138701_RCCBMS-00163-T_v1_RNA_OnPrem,NaN,Male,...,31.75620,30.75149,30.55896,30.17101,31.01545,21.45312,32.86620,34.24375,34.86720,32.73361


In [13]:
Y = []
X = []
for patient in range(len(data)): # Recorro pacientes
  print('patient: ' +str(patient))
  final_matrix = pd.read_csv('Patients/final_matrix_'+str(patient)+'.csv')
  if data.PFS[patient] < 3:
    Y.append(0)
  else:
    Y.append(1)
  if 'Unnamed: 0' in final_matrix.columns:
    final_matrix = final_matrix.drop(columns=['Unnamed: 0'])
#  print(final_matrix)
  X.append(final_matrix.to_numpy().flatten())
#print(X[0])
print(Y)

patient: 0
patient: 1
patient: 2
patient: 3
patient: 4
patient: 5
patient: 6
patient: 7
patient: 8
patient: 9
patient: 10
patient: 11
patient: 12
patient: 13
patient: 14
patient: 15
patient: 16
patient: 17
patient: 18
patient: 19
patient: 20
patient: 21
patient: 22
patient: 23
patient: 24
patient: 25
patient: 26
patient: 27
patient: 28
patient: 29
patient: 30
patient: 31
patient: 32
patient: 33
patient: 34
patient: 35
patient: 36
patient: 37
patient: 38
patient: 39
patient: 40
patient: 41
patient: 42
patient: 43
patient: 44
patient: 45
patient: 46
patient: 47
patient: 48
patient: 49
patient: 50
patient: 51
patient: 52
patient: 53
patient: 54
patient: 55
patient: 56
patient: 57
patient: 58
patient: 59
patient: 60
patient: 61
patient: 62
patient: 63
patient: 64
patient: 65
patient: 66
patient: 67
patient: 68
patient: 69
patient: 70
patient: 71
patient: 72
patient: 73
patient: 74
patient: 75
patient: 76
patient: 77
patient: 78
patient: 79
patient: 80
patient: 81
patient: 82
patient: 83
pa

In [14]:
# X[0].to_numpy().flatten()
X[0]

array([0., 0., 0., ..., 0., 0., 0.])

### 3. Train-Test dataset split

In [15]:
XTrain, XTest, yTrain, yTest = train_test_split(X, Y, test_size=0.2, random_state=125, stratify=Y)

print('Training set size:', len(XTrain))
print('Target column size of the training set:', len(yTrain))
print('Test set size:', len(XTest))
print('Target column size of the test set:', len(yTest))

Training set size: 144
Target column size of the training set: 144
Test set size: 37
Target column size of the test set: 37


In [16]:
param_grid = {'max_iter': [200, 500, 1000],
            'activation': ['identity', 'logistic', 'tanh', 'relu'],
            'random_state': [125],
            'max_fun': [300, 500,1000, 5000, 10000, 15000, 20000],
            'hidden_layer_sizes': [3,5],
            'solver': ['lbfgs']}

# I created a GridSearchCV which allows us to systematically evaluate and select the parameters of our model.
# By indicating a model and the parameters to test, you can evaluate the performance of the first one based on the
# seconds through cross validation.
clf = GridSearchCV(MLPClassifier(), param_grid, cv =5)

clf.fit(XTrain , yTrain)


C:\Users\sandr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\sandr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, sel

KeyboardInterrupt: 

In [11]:
print("Best estimate of parameters according to GridSearchCV:")
model = clf.best_estimator_
# Fit the model with the best parameters
model.fit(XTrain , yTrain)

Best estimate of parameters according to GridSearchCV:


MLPClassifier(activation='logistic', hidden_layer_sizes=3, max_fun=1000,
              max_iter=500, random_state=125, solver='lbfgs')

In [12]:
print("Best result of the cross validation of the model with the best paramters:" +str(clf.best_score_))

Best result of the cross validation of the model with the best paramters:0.5492610837438423


### 5. Prediction

In [13]:
# Making predictions with the optimal model on the training dataset
yhatTrain = model.predict(XTrain)
contTrain = 0

# Comparing with the Target column and check how many hits there have been
for i in range(0,len(yTrain),1) :
    if (yhatTrain[i] == yTrain[i]):
        contTrain = contTrain + 1

In [14]:
# Making predictions with the optimal model on the test dataset
yhatTest = model.predict(XTest)
contTest = 0

# Comparing with the Target column and check how many hits there have been
for i in range(0,len(yTest),1) :
    if (yhatTest[i] == yTest[i]):
        contTest = contTest + 1


### 6. Results

In [15]:
print('Final accuracy on the training dataset:' + str(contTrain/len(yTrain)))
print('Final accuracy on the testing dataset: ' + str(contTest/len(yTest)))

Final accuracy on the training dataset:0.5277777777777778
Final accuracy on the testing dataset: 0.5405405405405406


In [16]:
from sklearn.metrics import classification_report,confusion_matrix

print('----------------Confusion Matrix (Training)------------------')
print(confusion_matrix(yTrain,yhatTrain))
print('Input data:  ' + str(np.array(yTrain)))
print('Prediction:        ' +str(yhatTrain))

----------------Confusion Matrix (Training)------------------
[[ 0 68]
 [ 0 76]]
Input data:  [1 0 1 0 1 1 0 0 1 0 1 0 1 0 1 1 0 1 1 1 0 0 0 1 0 0 0 0 0 1 1 1 1 1 0 1 1
 0 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 1 0 1 1 0 0 0 0 1 1 1 0 1 0 0
 0 1 0 1 0 0 0 1 1 0 0 1 1 1 1 1 0 1 0 1 0 0 1 0 1 0 0 1 0 0 1 1 1 1 0 1 1
 1 1 0 1 0 0 0 1 0 1 0 1 0 0 1 1 1 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0 1]
Prediction:        [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [17]:
print(classification_report(yTrain,yhatTrain))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        68
           1       0.53      1.00      0.69        76

    accuracy                           0.53       144
   macro avg       0.26      0.50      0.35       144
weighted avg       0.28      0.53      0.36       144



C:\Users\sandr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sandr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sandr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1308: Unde

In [18]:
print('----------------Confusion Matrix (Test)------------------')
print(confusion_matrix(yTest,yhatTest))
print('Input data:  ' + str(np.array(yTest)))
print('Prediction:        ' +str(yhatTest))

----------------Confusion Matrix (Test)------------------
[[ 0 17]
 [ 0 20]]
Input data:  [0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 0 0 0 1 1 0 0 1 0 1 1 0 0 0 1 1 0 1 0]
Prediction:        [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [19]:
print(classification_report(yTest,yhatTest)) 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.54      1.00      0.70        20

    accuracy                           0.54        37
   macro avg       0.27      0.50      0.35        37
weighted avg       0.29      0.54      0.38        37



C:\Users\sandr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sandr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sandr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1308: Unde